# Train upper_base + Evaluation

In [ ]:
!pip install -q kaggle

In [ ]:
pip install 'h5py==2.10.0' --force-reinstall

  Using cached h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached numpy-1.21.2-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
ERROR: Operation cancelled by user


In [ ]:
!pip install ISR

  Using cached gast-0.2.2-py3-none-any.whl
  Using cached tensorboard-2.0.2-py3-none-any.whl (3.8 MB)
  Using cached tensorflow_estimator-2.0.1-py2.py3-none-any.whl (449 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.13.0 requires gast>=0.3.2, but you have gast 0.2.2 which is incompatible.
tensorflow-gpu 2.6.0 require

In [ ]:
!pip install --upgrade tensorflow-gpu
!pip install --upgrade keras

  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached tensorflow_estimator-2.6.0-py2.py3-none-any.whl (462 kB)
  Using cached tensorboard-2.6.0-py3-none-any.whl (5.6 MB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.0.1
    Uninstalling tensorflow-estimator-2.0.1:
      Successfully uninstalled tensorflow-estimator-2.0.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.0.2
    Uninstalling tensorboard-2.0.2:
      Successfully uninstalled tensorboard-2.0.2
  Attempting uninstall: gast
    Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.0.0 requires gast==0.2.2, but you have gast 0.4.0 which is incompatible.
tensorflow 2.0.0 requires tensor

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d congthjnh/aiijc-new128x128

 99% 918M/930M [00:21<00:00, 41.6MB/s]
100% 930M/930M [00:21<00:00, 46.2MB/s]


In [ ]:
!unzip -qq "/content/aiijc-new128x128.zip" -d "/content/aiijc-new128x128 (train)"

In [ ]:
from IPython.display import Image, display
import numpy as np
import os
import tensorflow
import math
import random
from PIL import ImageFile, Image
import pandas as pd
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import *
import matplotlib.pyplot as plt
import re
from tqdm import tqdm
import pickle
import tensorflow as tf
from tensorflow.keras.applications import *
from tensorflow.keras import *
import tensorflow as tf 
import cv2
ImageFile.LOAD_TRUNCATED_IMAGES = True
%matplotlib inline
from shutil import copyfile
import glob
import imageio
import PIL
import time

In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time

from IPython import display

In [ ]:
root_dir = "/content/aiijc-new128x128 (train)/New folder"
class_names = os.listdir(root_dir)
class_names.sort()
class_names.pop()

'unknown'

In [ ]:
x = []
y = []
z = []
X_train_paths = []
X_val_paths = []
y_val = []

for label in tqdm(class_names):
  img_names = os.listdir(root_dir + "/" + label)
  count = 0
  while count < 5:
    x.append(os.path.join(label, img_names[count]))
    y.append("sign")
    X_val_paths.append(os.path.join(label, img_names[count]))
    y_val.append(1)
    z.append("val")
    img_names.pop(0)
    count += 1
  for img_name in img_names:
    x.append(os.path.join(label, img_name))
    y.append("sign")
    X_train_paths.append(os.path.join(os.path.join(label, img_name)))
    z.append("train")

unknown_class = os.listdir(root_dir + "/" + "unknown")
for i in tqdm(range(len(unknown_class))):
  x.append(os.path.join("unknown", unknown_class[i]))
  y.append("unknown")
  X_val_paths.append(os.path.join("unknown", unknown_class[i]))
  y_val.append(0)
  z.append("val")

100%|██████████| 971/971 [00:00<00:00, 250194.69it/s]


In [ ]:
image_size = 64

def read_and_prep_images(img_paths, img_height = image_size, img_width = image_size):
    imgs = [load_img(os.path.join(root_dir,img_path), target_size = (img_height, img_width)) for img_path in tqdm(img_paths)]
    img_array = np.array([img_to_array(img) for img in imgs])
    output = img_array/255.0
    return(output)

X_train = read_and_prep_images(X_train_paths)
np.random.shuffle(X_train)
X_test = read_and_prep_images(X_val_paths)

100%|██████████| 1876/1876 [00:01<00:00, 1191.70it/s]


In [ ]:
BUFFER_SIZE = 45000
BATCH_SIZE = 64
train_dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
from ISR.models import RRDN
from ISR.models import Discriminator
from ISR.models import Cut_VGG19

lr_train_patch_size = 32
layers_to_extract = [5, 9]
scale = 2
hr_train_patch_size = lr_train_patch_size * scale

rrdn  = RRDN(arch_params = {'C':4, 'D':3, 'G':64, 'G0':64, 'T':10, 'x':scale}, patch_size=lr_train_patch_size)
f_ext = Cut_VGG19(patch_size = hr_train_patch_size, layers_to_extract = layers_to_extract)
discr = Discriminator(patch_size = hr_train_patch_size, kernel_size = 3)

80150528/80134624 [==============================] - 1s 0us/step


In [ ]:
rrdn.model.load_weights('/content/drive/MyDrive/Geoservice analysis/ISR/generator(4).hdf5')

In [ ]:
discr.model.load_weights('/content/drive/MyDrive/Geoservice analysis/ISR/discriminator(4).hdf5')

In [ ]:
gen_base = load_model('/content/drive/MyDrive/Geoservice analysis/ISR/gen_base.h5')
discr_base = load_model('/content/drive/MyDrive/Geoservice analysis/ISR/discr_base.h5')

In [ ]:
rrdn.model.trainable = False

generator = Sequential([
    gen_base,
    rrdn.model                   
])

In [ ]:
discr.model.trainable = False

discriminator = Sequential([
    discr.model,
    discr_base              
])

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 64

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
@tf.function #converts regular python code to a callable Tensorflow graph function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim]) #noise dim is the latent representation
    print('Done noise')

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape: #one for generator, the other for discriminator
      generated_images = generator(noise, training = True) #generate a batch of fake images (from noise)
      print('Done generated images')

      real_output = discriminator(images, training = True)
      fake_output = discriminator(generated_images, training = True)
      print('Done fake output')

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

      print('Done loss')

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    #gen loss is target, generator is the source (gradient tape will record all of the operatirons happen to peform backward gradient)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    print('Done gradients')

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    print('Done opt')

In [ ]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    print('Start-timing')

    for image_batch in dataset:
      train_step(image_batch)
      print('1 step')
    
    print('---Done 1 epoch---')

    # Produce images for the GIF as you go
    # Save the model every 1 epochs
    gen_base.save('/content/drive/MyDrive/Geoservice analysis/gen_base.h5')
    discr_base.save('/content/drive/MyDrive/Geoservice analysis/discr_base.h5')
    generator.save('/content/drive/MyDrive/Geoservice analysis/big_generator.h5')
    discriminator.save('/content/drive/MyDrive/Geoservice analysis/big_discriminator.h5')

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

      # Generate after the final epoch
  display.clear_output(wait = True)
  generate_and_save_images(generator,
                           epochs,
                           seed
                           )

In [ ]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize = (8, 8))

  for i in range(predictions.shape[0]):
      plt.subplot(8, 8, i + 1)
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5)
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

In [ ]:
train(train_dataset, 5)

Start-timing
Done noise
Done generated images
Done fake output
Done loss
Done gradients
Done opt
1 step
1 step
1 step
1 step
1 step
1 step
1 step
1 step
1 step
1 step
1 step
1 step
1 step
1 step
1 step
1 step
1 step
1 step
1 step
1 step


In [ ]:
discr_base_val = load_model('/content/drive/MyDrive/Geoservice analysis/discr_base.h5')

In [ ]:
discriminator_val = Sequential([
    discr.model,
    discr_base_val  
])

In [ ]:
import sklearn
from sklearn.metrics import roc_auc_score

In [ ]:
y_pred= discriminator_val.predict(X_test)

In [ ]:
print(roc_auc_score(y_val, y_pred))

# Train ISR base

In [ ]:
!kaggle datasets download -d congthjnh/encoded-imgs

In [ ]:
!unzip -qq "/content/encoded-imgs.zip" -d "/content/aiijc-80x80 (train)"

In [ ]:
df = pd.read_csv('/content/train.csv')

In [ ]:
df_uk = df.loc[df['label']=='unknown']

In [ ]:
for i in tqdm(df_uk['filename']):
  df_uk.loc[df['filename']==i, 'filename']= i.replace('train_images/','')

In [ ]:
for i in tqdm(df_uk['filename']):
  os.remove('/content/aiijc-80x80 (train)/Autoencoder/'+i)

In [ ]:
os.makedirs('aiijc-80x80 (validation)', exist_ok = True)

In [ ]:
for i in tqdm(random.sample(os.listdir('/content/aiijc-80x80 (train)/Autoencoder'), k = math.floor(len(os.listdir('/content/aiijc-80x80 (train)/Autoencoder'))/30))):
  copyfile('/content/aiijc-80x80 (train)/Autoencoder/' + i, '/content/aiijc-80x80 (validation)/' + i)
  os.remove('/content/aiijc-80x80 (train)/Autoencoder/' + i)


In [ ]:
def preprocess_img_init(item, name):
  if name == 'train':
    img = load_img(f'/content/aiijc-80x80 ({name})/Autoencoder/' + item)
    img_array = img_to_array(img)
    img_resized = tf.image.resize(img_array, size = [32, 32], preserve_aspect_ratio = True)
    save_img(f'/content/aiijc-32x32 ({name})/' + item, x = img_resized)
  elif name == 'validation':
    img = load_img(f'/content/aiijc-80x80 ({name})/' + item)
    img_array = img_to_array(img)
    img_resized = tf.image.resize(img_array, size = [32, 32], preserve_aspect_ratio = True)
    save_img(path = f'/content/aiijc-32x32 ({name})/' + item, x = img_resized)   

In [ ]:
os.makedirs('aiijc-32x32 (validation)', exist_ok = True) 
os.makedirs('aiijc-32x32 (train)', exist_ok = True)

In [ ]:
for i in tqdm(os.listdir('/content/aiijc-80x80 (train)/Autoencoder')):
  preprocess_img_init(item = i, name = 'train')

for i in tqdm(os.listdir('/content/aiijc-80x80 (validation)')):
  preprocess_img_init(item = i, name = 'validation')

In [ ]:
os.makedirs('/content/aiijc-64x64 (train)', exist_ok = True)
os.makedirs('/content/aiijc-64x64 (validation)', exist_ok = True)

In [ ]:
def preprocess_img(item, name):
  if name =='train':
    img = load_img(f'/content/aiijc-32x32 ({name})/' + item)
    img_array = img_to_array(img)
    img_resized = tf.image.resize(img_array, size = [64, 64], preserve_aspect_ratio = True)
    save_img(f'/content/aiijc-64x64 ({name})/' + item, x = img_resized)
  elif name =='validation':
    img = load_img(f'/content/aiijc-32x32 ({name})/' + item)
    img_array = img_to_array(img)
    img_resized = tf.image.resize(img_array, size = [64, 64], preserve_aspect_ratio = True)
    save_img(path = f'/content/aiijc-64x64 ({name})/' + item, x = img_resized)    

In [ ]:
for i in tqdm(os.listdir('/content/aiijc-32x32 (train)')):
  preprocess_img(item = i, name = 'train')

In [ ]:
for i in tqdm(os.listdir('/content/aiijc-32x32 (validation)')):
  preprocess_img(item = i, name = 'validation')

In [ ]:
from ISR.models import RRDN
from ISR.models import Discriminator
from ISR.models import Cut_VGG19

lr_train_patch_size = 32
layers_to_extract = [5, 9]
scale = 2
hr_train_patch_size = lr_train_patch_size * scale

rrdn  = RRDN(arch_params = {'C':4, 'D':3, 'G':64, 'G0':64, 'T':10, 'x':scale}, patch_size = lr_train_patch_size)
f_ext = Cut_VGG19(patch_size = hr_train_patch_size, layers_to_extract = layers_to_extract)
discr = Discriminator(patch_size = hr_train_patch_size, kernel_size = 3)

In [ ]:
from shutil import rmtree

rmtree('aiijc-80x80 (train)')
rmtree('aiijc-80x80 (validation)')

In [ ]:
from ISR.train import Trainer
loss_weights = {
  'generator': 0.0,
  'feature_extractor': 0.0833,
  'discriminator': 0.01
}
losses = {
  'generator': 'mae',
  'feature_extractor': 'mse',
  'discriminator': 'binary_crossentropy'
}

log_dirs = {'logs': './logs', 'weights': './weights'}

learning_rate = {'initial_value': 0.0004, 'decay_factor': 0.5, 'decay_frequency': 3}

flatness = {'min': 0.0, 'max': 0.15, 'increase': 0.01, 'increase_frequency': 5}

trainer = Trainer(
    generator = rrdn,
    discriminator = discr,
    feature_extractor = f_ext,
    lr_train_dir = '/content/aiijc-32x32 (train)',
    hr_train_dir = '/content/aiijc-64x64 (train)',
    lr_valid_dir = '/content/aiijc-32x32 (validation)',
    hr_valid_dir = '/content/aiijc-64x64 (validation)',
    loss_weights = loss_weights,
    learning_rate = learning_rate,
    flatness = flatness,
    dataname = 'image_dataset',
    log_dirs = log_dirs,
    weights_generator = '/content/drive/MyDrive/Geoservice analysis/ISR/generator(3).hdf5',
    weights_discriminator = '/content/drive/MyDrive/Geoservice analysis/ISR/discriminator(3).hdf5',
    n_validation = 100,
)

In [ ]:
trainer.train(
    epochs = 7,
    steps_per_epoch = 1220,
    batch_size = 38,
    monitored_metrics = {'val_PSNR_Y': 'max'}
)

In [ ]:
gen_model = rrdn.model
gen_model.save('/content/drive/MyDrive/Geoservice analysis/ISR/generator(4).hdf5')
discr_model = discr.model
discr_model.save('/content/drive/MyDrive/Geoservice analysis/ISR/discriminator(4).hdf5')